In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from fastai.vision.all import *
import torchaudio

In [ ]:
tp_train = pd.read_csv("../input/rfcx-species-audio-detection/train_tp.csv")
tp_train.head()

In [ ]:
DATA_PATH = Path('/kaggle/input/rfcx-species-audio-detection/')
train_files = get_files(DATA_PATH/'train', '.flac')

In [ ]:
# fixed_sample_rate = 22050

In [ ]:
tfms = [torchaudio.transforms.MelSpectrogram(n_mels=32),
        torchaudio.transforms.AmplitudeToDB()]

In [ ]:
dls = DataBlock(blocks=(TransformBlock(type_tfms=tfms), MultiCategoryBlock),                   
                 get_x=lambda x: torchaudio.load(x)[0],
                 get_y=lambda x: set(tp_train[tp_train.recording_id == x.stem].species_id) or {24}).dataloaders(train_files, bs=4)

In [ ]:
# sample, sample_rate = torchaudio.load(train_files[7])
# sample = tfms[0](sample)
# sample = tfms[1](sample)

In [ ]:
learn = cnn_learner(dls, resnet50, config={"n_in":1}, metrics=accuracy_multi).to_fp16()

In [ ]:
learn.fine_tune(7, 1e-2)

In [ ]:
sample = pd.read_csv(DATA_PATH/'sample_submission.csv')
test_files = [DATA_PATH/'test'/f'{id}.flac' for id in sample.recording_id]

In [ ]:
test_dl = dls.test_dl(test_files)
preds,_ = learn.get_preds(dl=test_dl)

In [ ]:
submission = pd.concat([sample['recording_id'], pd.DataFrame(preds[:,:-1].numpy(), columns=sample.columns[1:])], axis=1)
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)